#Importing libraries

In [ ]:
!pip install langdetect googletrans

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=1c8f6d0db06917d6721dd4192ad9b155d231899cb616377ffaf2639e9f506c03
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [ ]:
import pandas as pd
import numpy as np
import unicodedata
import re
from langdetect import detect, LangDetectException
from googletrans import Translator
from typing import Optional
import asyncio
import math

# Cleaning data

## Load candidate dataset

In [ ]:
url_candidate = "https://drive.google.com/file/d/1ZQGi12NYI7BQkjWKiTUf5GQCDkX64XfV/view?usp=sharing"
file_id_candidate = url_candidate.split('/d/')[1].split('/')[0]
csv_url_candidate = f'https://drive.google.com/uc?id={file_id_candidate}'
uncleaned_candidate_df = pd.read_csv(csv_url_candidate)
uncleaned_candidate_df.drop("Unnamed: 0", axis=1, inplace=True)
uncleaned_candidate_df

,user_url,userid,user_name,industry,desired_job,workplace_desired,current_place_of_residence,province,district,desired_salary,gender,marriage,age,work_experience,career_goals,skills
0,https://timviec365.vn/uvtv2/dao-huu-tai-uv1111...,1111684868,ĐÀO HỮU TÀI,['KD bất động sản'],Chuyên viên Kinh doanh Bất động sản,['Hà Nội'],NaN,Hà Nội,NaN,NaN,Nam,Độc thân,38.0,0 - 1 năm kinh nghiệm,?,?
1,https://timviec365.vn/uvtv2/ho-vinh-duc-uv199803,199803,Hồ Vĩnh Đức,['Xây dựng'],Kỹ sư kinh tế xây dựng,['Hồ Chí Minh'],Thành phố Thủ Đức - Hồ Chí Minh,Hồ Chí Minh,NaN,Thỏa thuận,Nam,?,37.0,NaN,-\tCông việc ổn định lâu dài\r\n-\tCó cơ hội ...,"-\tTrung thực, cẩn thận\r\n-\tQuan sát, Să..."
2,https://timviec365.vn/uvtv2/ngo-thi-bich-tram-...,1271594,Ngô thị bích trâm,['Khách sạn - Nhà hàng'],Nhân Viên nhà hàng,['Khánh Hòa'],Huyện Cam Lâm - Khánh Hòa,Khánh Hòa,NaN,Thỏa thuận,Nam,Độc thân,NaN,Chưa có kinh nghiệm,?,?
3,https://timviec365.vn/uvtv2/vo-ba-loi-uv160413,160413,Võ Bá Lợi,['Giao thông vận tải - Thủy lợi - Cầu đường'],đầu bếp,['Hồ Chí Minh'],Quận 12 - Hồ Chí Minh,Hồ Chí Minh,NaN,Thỏa thuận,Nam,Độc thân,NaN,Chưa có kinh nghiệm,Mong muốn tìm được chỗ làm ổn định lâu dài<br/>,Biết lắng nghe tiếp thu ý kiến của đồng nghiệp...
4,https://timviec365.vn/uvtv2/vo-thi-thuy-van-uv...,1111646039,Võ Thị thuỳ vân,['Quản trị kinh doanh'],Thực tập sinh kinh doanh,['Đồng Nai'],NaN,Đồng Nai,NaN,Thỏa thuận,Nữ,Độc thân,21.0,Chưa có kinh nghiệm,?,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46119,https://timviec365.vn/uvtv2/nguyen-thi-huong-u...,318287,Nguyễn Thị Hương Uyên,['Hành chính - Văn phòng'],hành chính văn phòng,['Hồ Chí Minh'],Quận 9 - Hồ Chí Minh,Hồ Chí Minh,NaN,Thỏa thuận,Nam,Độc thân,NaN,Hơn 5 năm kinh nghiệm,"- Có một công việc có nhiều cơ hội thăng tiến,...","-\tKỹ năng giao tiếp linh hoạt, khả năng thuyế..."
46120,https://timviec365.vn/uvtv2/le-van-linh-uv1111...,1111355647,lê văn linh,['An toàn lao động'],nhân viên hse,['Thừa Thiên Huế'],NaN,Thừa Thiên Huế,NaN,Thỏa thuận,Nam,Độc thân,23.0,0 - 1 năm kinh nghiệm,?,?
46121,https://timviec365.vn/uvtv2/cao-son-nguyen-uv1...,1360987,Cao Sơn Nguyên,['Nhân viên kinh doanh'],NHÂN VIÊN KINH DOANH(Đang tìm việc),['Hà Nội'],Quận Hoàng Mai - Hà Nội,Hà Nội,NaN,Từ 25.000.000 VNĐ,Nam,Đã kết hôn,29.0,Hơn 10 năm kinh nghiệm,?,?
46122,https://timviec365.vn/uvtv2/le-kim-nguyen-uv11...,1111343879,LÊ KIM NGUYÊN,['Nhân viên kinh doanh'],Nhân viên kinh doanh,['Tây Ninh'],NaN,Tây Ninh,NaN,Thỏa thuận,Nữ,Độc thân,27.0,Hơn 2 năm kinh nghiệm,?,?


## Dropping duplicates and unnecessary columns

In [ ]:
uncleaned_candidate_df.replace("?", np.nan, inplace=True)
uncleaned_candidate_df.drop_duplicates(inplace=True)
uncleaned_candidate_df.drop(["current_place_of_residence", "province", "district", "skills", "career_goals"], axis=1, inplace=True)

## Normalizing text

In [ ]:
def normalize_and_format(text: str) -> str:
    if not isinstance(text, str):
        return text
    # Normalize
    text = re.sub(r'<[^>]+>', ' ', text)
    text = re.sub(r'[\n\r\t\xa0]', ' ', text)        # Replace newline, carriage return, tab, non-breaking space with space
    text = re.sub(r'[\'\"\[\]]', '', text)           # Remove quotes and brackets
    text = re.sub(r' {2,}', ' ', text)               # Collapse multiple spaces
    text = text.strip()
    # Format
    return ' '.join(word.capitalize() for word in text.lower().split())

In [ ]:
columns_to_clean = ["user_name", "industry", "desired_job", "workplace_desired", "desired_salary", "gender", "marriage", "work_experience"]  # Example column names

for col in columns_to_clean:
    uncleaned_candidate_df[col] = uncleaned_candidate_df[col].apply(lambda x: normalize_and_format(x))

## Language detecting and translating to Vietnamese

In [ ]:
async def translate_to_vietnamese_if_foreign(text, translator):
    """
    Translates text to Vietnamese if it's detected as a foreign language,
    but explicitly ignores (does not translate) text detected as English.

    Args:
        text (str): The input text to be checked and potentially translated.
        translator: An instance of your translation library's translator.

    Returns:
        str: The translated text (if a non-English foreign language was detected),
             or the original text if it was Vietnamese, English, or an error occurred/detection failed.
    """
    if not isinstance(text, str) or not text.strip():
      print(f"Input '{text}' is not a non-empty string. Returning as is.")
      return text

    original_text_for_logging = text

    try:
        lang = detect(text)
        print(f"Input: '{original_text_for_logging}', Detected language: '{lang}'")

        if lang == 'vi':
            print(f"Language detected as Vietnamese for '{original_text_for_logging}'. No translation needed.")
            return text
        elif lang == 'en':
            print(f"Language detected as English for '{original_text_for_logging}'. Returning original text as per instruction.")
            return text
        else:  # Language is foreign and not English
            print(f"Detected non-English foreign language: {lang} for '{original_text_for_logging}'. Translating to Vietnamese...")
            translation = await translator.translate(text, dest='vi')
            translated_text = translation.text
            print(f"Original: '{original_text_for_logging}', Translated by API: '{translated_text}'")
            return translated_text

    except LangDetectException:
        # If language detection fails, it's often for short, ambiguous, or technical (English) terms.
        # Given the new requirement, it's safer to return the original text.
        print(f"Could not detect language for '{original_text_for_logging}'. Returning original text.")
        return original_text_for_logging

    except Exception as e_main:
        print(f"An unexpected error occurred while processing '{original_text_for_logging}': {e_main}. Returning original text.")
        return original_text_for_logging

async def translate_column_async(df, column_name, translator):
    """
    Asynchronously translates a specified column of a DataFrame using the
    modified translate_to_vietnamese_if_foreign function.
    """
    tasks = []
    for text_content in df[column_name]:
        task = asyncio.create_task(translate_to_vietnamese_if_foreign(text_content, translator))
        tasks.append(task)
    translated_texts = await asyncio.gather(*tasks)
    return translated_texts

In [ ]:
async def main_translation_logic():
    print("Initializing translator...")
    translator = Translator()

    print(f"Starting translation for 'desired_job' column with {len(uncleaned_candidate_df)} rows...")
    # Await the function that gathers all translation tasks
    translated_texts = await translate_column_async(uncleaned_candidate_df, 'desired_job', translator)

    # Now assign the results back to your DataFrame
    uncleaned_candidate_df['desired_job_translated'] = translated_texts
    print("Translation tasks complete.")
    print("First 5 rows of the 'desired_job_translated' column:")
    print(uncleaned_candidate_df[['desired_job', 'desired_job_translated']].head())

# To run the async function in a Colab cell:
await main_translation_logic()

Streaming output truncated to the last 5000 lines.
Language detected as Vietnamese for 'Nhân Viên Lái Xe Bằng B2'. No translation needed.
Input: 'Nhân Viên Kinh Doanh', Detected language: 'vi'
Language detected as Vietnamese for 'Nhân Viên Kinh Doanh'. No translation needed.
Input: 'Kỹ Sư Điều Khiển(đang Tìm Việc)', Detected language: 'vi'
Language detected as Vietnamese for 'Kỹ Sư Điều Khiển(đang Tìm Việc)'. No translation needed.
Input: 'Nhân Viên Qcp(đang Tìm Việc)', Detected language: 'vi'
Language detected as Vietnamese for 'Nhân Viên Qcp(đang Tìm Việc)'. No translation needed.
Input: 'Nhân Viên Bảo Vệ', Detected language: 'vi'
Language detected as Vietnamese for 'Nhân Viên Bảo Vệ'. No translation needed.
Input: 'Trợ Lý', Detected language: 'vi'
Language detected as Vietnamese for 'Trợ Lý'. No translation needed.
Input: 'Nhân Viên Hành Chính Văn Phòng', Detected language: 'vi'
Language detected as Vietnamese for 'Nhân Viên Hành Chính Văn Phòng'. No translation needed.
Input: 'Orth

## Normalizing salary and work experience

### Salary

In [ ]:
def parse_and_categorize_salary(salary_text):
    if salary_text is None or isinstance(salary_text, float):
        return "Thỏa Thuận"

    salary_text = salary_text.strip()

    if "Thỏa Thuận" in salary_text:
        return "Thỏa Thuận"

    s = salary_text.replace(",", ".").lower()
    s = re.sub(r"\s+", " ", s)
    s = s.replace("vnđ", "").replace("vnd", "")
    s = s.replace("usd", "*23000") # Assuming 1 USD = 23000 VND
    s = s.replace("triệu", "*1000000")
    s = s.replace("đ", "")
    s = s.replace("từ", "").strip()
    s = s.replace("–", "-") # Handle en-dash

    # Convert negative signs to positive (for inputs like "-10 đến -15")
    s = re.sub(r"-(\d)", r"\1", s)

    def safe_eval(value_str):
        # remove any extra characters that are not digits, operations or decimal points for eval
        processed_value_str = "".join(filter(lambda x: x.isdigit() or x in ['.', '*', '-', '+', '/'], value_str))
        if not any(op in processed_value_str for op in ['*', '-', '+', '/']): # if no operation, assume it's millions if no explicit multiplier
             # check if it's a number that might be a direct value without "triệu"
            try:
                num = float(processed_value_str)
                if num < 1000: # Heuristic: if it's a small number, assume millions
                    processed_value_str = f"{processed_value_str}*1000000"
                else: # larger number, assume it's already in VND
                    pass
            except ValueError: # Not a simple number, might have other characters, rely on original logic or let eval fail
                 if "*" not in value_str : # Fallback to original logic for non-numeric simple strings
                    processed_value_str = f"{value_str}*1000000"


        # Ensure eval is safe by only allowing simple arithmetic
        # This is a basic check; for production, a more robust parser is better than eval.
        if not re.match(r'^[\d\.\s\*\+\-\/]+$', processed_value_str):
            raise ValueError("Invalid characters in expression for eval")
        return int(eval(processed_value_str))


    # Handle "Trên X"
    if s.startswith("trên"):
        s_val = s.replace("trên", "").strip()
        try:
            min_val = safe_eval(s_val)
            return categorize_by_range(f"{min_val} - ∞")
        except:
            return "Thỏa Thuận"

    # Handle "Đến X"
    if s.startswith("đến"):
        s_val = s.replace("đến", "").strip()
        try:
            max_val = safe_eval(s_val)
            return categorize_by_range(f"0 - {max_val}")
        except:
            return "Thỏa Thuận"

    # Handle "X - Y"
    if "-" in s:
        parts = s.split("-")
        if len(parts) == 2:
            try:
                min_val_str = parts[0].strip()
                max_val_str = parts[1].strip()

                # If max_val_str is just a number like "15" and min_val_str has "triệu" like "10 triệu"
                # then "15" should also be "15 triệu"
                if "000000" in str(safe_eval(min_val_str)) and "*" not in max_val_str and float(max_val_str.replace(',','.')) < 1000 :
                     max_val_str = f"{max_val_str}*1000000"


                min_val = safe_eval(min_val_str)
                max_val = safe_eval(max_val_str)

                if min_val > max_val : # Swap if min > max (e.g. "15-10 triệu")
                    min_val, max_val = max_val, min_val

                return categorize_by_range(f"{min_val} - {max_val}")
            except:
                return "Thỏa Thuận"
        else: # If more than one '-' might be malformed
            return "Thỏa Thuận"


    # Handle single value
    try:
        val = safe_eval(s)
        return categorize_by_range(f"{val} - {val}")
    except:
        return "Thỏa Thuận"

def categorize_by_range(salary_range_str):
    if salary_range_str is None or salary_range_str == "Thỏa Thuận":
        return "Thỏa Thuận"

    try:
        min_str_part, max_str_part = salary_range_str.split(" - ")
        min_val_parsed = int(min_str_part)

        is_max_infinite_parsed = (max_str_part == "∞")
        max_val_parsed_numeric = float('inf') if is_max_infinite_parsed else int(max_str_part)

        # Case 1: Input was "Đến X", parsed as "0 - X"
        if min_val_parsed == 0 and not is_max_infinite_parsed:
            if max_val_parsed_numeric > 40000000:
                return "Trên 40,000,000"
            elif max_val_parsed_numeric > 20000000: # Max is (20M, 40M]
                return "20,000,000 - 40,000,000"
            elif max_val_parsed_numeric > 15000000: # Max is (15M, 20M]
                return "15,000,000 - 20,000,000"
            elif max_val_parsed_numeric > 10000000: # Max is (10M, 15M]
                return "10,000,000 - 15,000,000"
            elif max_val_parsed_numeric > 5000000:  # Max is (5M, 10M]
                return "5,000,000 - 10,000,000"
            elif max_val_parsed_numeric >= 1000000: # Max is [1M, 5M]
                return "1,000,000 - 5,000,000"
            else: # Max is < 1M
                return "1,000,000 - 5,000,000" # Default to lowest band

        # Case 2: Standard ranges, "Trên X", or single values
        # Rule for "Trên 40,000,000"
        if min_val_parsed > 40000000 or is_max_infinite_parsed or \
           (not is_max_infinite_parsed and max_val_parsed_numeric > 40000000):
            return "Trên 40,000,000"

        # Bucket based on min_val_parsed (max_val_parsed_numeric is <= 40M and finite here)
        if min_val_parsed >= 20000000: # Range [20M, 40M]
            return "20,000,000 - 40,000,000"
        elif min_val_parsed >= 15000000: # Range [15M, <20M)
            return "15,000,000 - 20,000,000"
        elif min_val_parsed >= 10000000: # Range [10M, <15M)
            return "10,000,000 - 15,000,000"
        elif min_val_parsed >= 5000000:  # Range [5M, <10M)
            return "5,000,000 - 10,000,000"
        elif min_val_parsed >= 1000000:   # Range [1M, <5M)
            return "1,000,000 - 5,000,000"
        else: # min_val_parsed < 1,000,000
            return "1,000,000 - 5,000,000" # Default to the lowest band

    except Exception:
        return "Thỏa Thuận"


In [ ]:
uncleaned_candidate_df["desired_salary"] = uncleaned_candidate_df["desired_salary"].apply(parse_and_categorize_salary)
uncleaned_candidate_df["desired_salary"].unique()

array(['Thỏa Thuận', '1,000,000 - 5,000,000', '5,000,000 - 10,000,000',
       '10,000,000 - 15,000,000', 'Trên 40,000,000',
       '15,000,000 - 20,000,000', '20,000,000 - 40,000,000'], dtype=object)

### Work Experience

In [ ]:
def normalize_experience(experience_text):
    """
    Normalizes experience categories to standardized year ranges in Vietnamese.

    Args:
        experience_text (str or float): The experience string to normalize.
                                        Can be float('nan').

    Returns:
        str: The normalized experience category.
    """
    if isinstance(experience_text, float) and math.isnan(experience_text):
        return "Không yêu cầu kinh nghiệm"

    if not isinstance(experience_text, str):
        return "Không yêu cầu kinh nghiệm"

    # Normalize the input string for consistent matching
    exp_text_lower = experience_text.lower().strip()

    # Handle specific cases based on your current categories
    if "chưa có" in exp_text_lower:  # "Chưa Có Kinh Nghiệm"
        return "Chưa Có Kinh Nghiệm"

    elif "0 - 1 năm" in exp_text_lower or "0-1 năm" in exp_text_lower:  # "0 - 1 Năm Kinh Nghiệm"
        return "Dưới 1 năm"

    elif "hơn 10 năm" in exp_text_lower:  # "Hơn 10 Năm Kinh Nghiệm"
        return "Trên 10 năm"

    elif "hơn 5 năm" in exp_text_lower:  # "Hơn 5 Năm Kinh Nghiệm"
        return "5-10 năm"

    elif "hơn 2 năm" in exp_text_lower:  # "Hơn 2 Năm Kinh Nghiệm"
        return "3-5 năm"

    elif "hơn 1 năm" in exp_text_lower:  # "Hơn 1 Năm Kinh Nghiệm"
        return "1-3 năm"

    else:
        # Default for any unrecognized format
        return "Chưa rõ"

In [ ]:
uncleaned_candidate_df["work_experience"] = uncleaned_candidate_df["work_experience"].apply(normalize_experience)
uncleaned_candidate_df["work_experience"].unique()

array(['Dưới 1 năm', 'Không yêu cầu kinh nghiệm', 'Chưa Có Kinh Nghiệm',
       '1-3 năm', '3-5 năm', 'Trên 10 năm', '5-10 năm'], dtype=object)

## Handling null values

In [ ]:
uncleaned_candidate_df['gender'].fillna('Khác', inplace=True)
uncleaned_candidate_df['marriage'].fillna('Khác', inplace=True)

<ipython-input-13-3681213489>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  uncleaned_candidate_df['gender'].fillna('Khác', inplace=True)
<ipython-input-13-3681213489>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)',

In [ ]:
uncleaned_candidate_df = uncleaned_candidate_df[pd.notna(uncleaned_candidate_df["desired_job_translated"])]

In [ ]:
uncleaned_candidate_df.drop("desired_job", axis=1, inplace=True)

<ipython-input-15-3441425809>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uncleaned_candidate_df.drop("desired_job", axis=1, inplace=True)


In [ ]:
uncleaned_candidate_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44963 entries, 0 to 46123
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   user_url                44963 non-null  object 
 1   userid                  44963 non-null  int64  
 2   user_name               44963 non-null  object 
 3   industry                44963 non-null  object 
 4   workplace_desired       44963 non-null  object 
 5   desired_salary          44963 non-null  object 
 6   gender                  44963 non-null  object 
 7   marriage                44963 non-null  object 
 8   age                     25952 non-null  float64
 9   work_experience         44963 non-null  object 
 10  desired_job_translated  44963 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 4.1+ MB


In [ ]:
uncleaned_candidate_df.to_csv("cleaned_candidate_data.csv")